In [1]:
from langchain_community.llms import Ollama
from langchain_community.embeddings.laser import LaserEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.retrievers import BM25Retriever
from langchain.chains.question_answering import load_qa_chain
from langchain_core.prompts.chat import ChatPromptTemplate
# from sentence_transformers import SentenceTransformer
from langchain_community.vectorstores import FAISS
import numpy as np

In [2]:
ollama_base_url='http://tempestade.facom.ufms.br:11435'

ollama = Ollama(base_url=ollama_base_url, model="phi3:medium")
# embeddings_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')
embeddings_model = LaserEmbeddings(lang='por_Latn')

2024-07-17 06:57:32,501 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2024-07-17 06:57:33,424 | INFO | laser_encoders.download_models |  - laser2.spm already downloaded
2024-07-17 06:57:33,559 | INFO | laser_encoders.download_models |  - laser2.pt already downloaded
2024-07-17 06:57:33,560 | INFO | laser_encoders.download_models |  - laser2.spm already downloaded
2024-07-17 06:57:33,561 | INFO | laser_encoders.download_models |  - laser2.cvocab already downloaded


In [49]:
# doc_path = './resultados/EDITAL-PROAES-N-70-DE-JUNHO-DE-2024_CADASTRO-PARA-BOLSA-PERMANENCIA-DO-MEC-PARA-ESTUDANTES-INDIGENAS-E-QUILOMBOLA-MA-GRADUACAO-PROGRAMA-BPMEC-2024.pdf'
doc_path = './resultados/EDITAL-PROAES-PROGRAD-PROPP.pdf'

pdf_loader = PyPDFLoader(doc_path)

loaded_pdf = pdf_loader.load()

text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=0,
        separators=['\n', ' ']
)

pages = text_splitter.split_documents(loaded_pdf)

In [50]:
vectorstore = FAISS.from_documents(
    pages,
    embeddings_model
)

In [51]:
prompt_structure = '''
        Baseado nos seguintes documentos:
        {context}
        Responda a pergunta abaixo:
        {query}
        '''

qa_prompt = ChatPromptTemplate.from_messages([
        ('system', '''Você é um funcionário da Universidade Federal de Mato Grosso do Sul que tem conhecimento
         de todo o documento apresentado como contexto e 
         responde todas as perguntas em Portugues do Brasil. Você responde a perguntas sobre o documento apresentado, usando o contexto fornecido.
         Use a seguinte estrutura para responder as perguntas:'''),
        ('human', '''Como será a lista de espera?'''),
        ('ai', '''De acordo com o item 3.4 do edital, a lista de espera será definida pela ordem de cadastro aprovado 
          e permanecerá para o atendimento por meio da liberação de novas vagas pelo MEC.'''),
        ('human', '''Qual o número mínimo de membros das comissões temporárias constituídas pelo Conselho?'''),
        ('ai', '''De acordo com o Art. 61, as comissões temporárias deverão ser constituídas por, no mínimo, três membros.'''),
        ('human', prompt_structure)
])

chain = load_qa_chain(ollama, chain_type='stuff', verbose=True, prompt=qa_prompt)

In [53]:
query = input()

In [60]:
retrieval_result = vectorstore.similarity_search_with_relevance_scores(query, k=10)

In [62]:
# get the page index of the page with the most similar chunk
page_idx = retrieval_result[0][0].metadata['page']

In [63]:
retrieval_result = np.concatenate(
    (
        [x for x in pages if x.metadata['page'] == page_idx],
        [page[0] for page in retrieval_result]),
    axis=0
)

In [64]:
output = chain.invoke(
        {'input_documents':retrieval_result, 'query':query}
)

print(output['output_text'])



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Você é um funcionário da Universidade Federal de Mato Grosso do Sul que tem conhecimento
         de todo o documento apresentado como contexto e 
         responde todas as perguntas em Portugues do Brasil. Você responde a perguntas sobre o documento apresentado, usando o contexto fornecido.
         Use a seguinte estrutura para responder as perguntas:
Human: Como será a lista de espera?
AI: De acordo com o item 3.4 do edital, a lista de espera será definida pela ordem de cadastro aprovado 
          e permanecerá para o atendimento por meio da liberação de novas vagas pelo MEC.
Human: Qual o número mínimo de membros das comissões temporárias constituídas pelo Conselho?
AI: De acordo com o Art. 61, as comissões temporárias deverão ser constituídas por, no mínimo, três membros.
Human: 
        Baseado nos seguintes documentos:
        Pró-Reitoria de Assuntos Estudantis - 